In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=3325d7076039d0d91806f63bda851c23625067dd3109081bfb28915a9d338975
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [64]:
from pyspark.sql import SparkSession

In [111]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [112]:
from pyspark.ml.regression import LinearRegression

In [113]:
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

In [114]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [115]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [116]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [117]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [118]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [120]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App",
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [121]:
output = assembler.transform(data)

In [122]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [123]:
final_data = output.select("features",'Yearly Amount Spent')

In [124]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [125]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                360|
|   mean|  497.4763683035563|
| stddev|   78.4349278735554|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [126]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [127]:
lrModel = lr.fit(train_data,)

In [128]:
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.77115961677442,38.96042665662066,0.10568912285090648,61.65394661782269] Intercept: -1043.9769760730694


In [129]:
test_results = lrModel.evaluate(test_data)

In [130]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-16.612999976080687|
| -4.649371923514025|
|   9.83589987319624|
|  19.66952976984686|
|  3.788537063645549|
| 1.3341137940894896|
| 3.0815992588972563|
| -3.166953083658484|
| -5.218397884008823|
|-14.572313339593165|
| -4.562718222397109|
|  6.825162665052574|
|-6.4276004237125335|
|-1.5140168261322628|
|-11.385744885222039|
|  8.040407666705903|
| -9.115928673334054|
| -17.12778723381831|
|-13.540484845186711|
|  -5.98121626748059|
+-------------------+
only showing top 20 rows



In [131]:
unlabeled_data = test_data.select('features')

In [132]:
predictions = lrModel.transform(unlabeled_data)

In [133]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8162006488763...| 282.6993409245497|
|[30.8364326747734...|472.15127235050363|
|[31.1695067987115...|417.52063092909657|
|[31.3123495994443...|443.92188825809376|
|[31.3584771924370...|491.38741338582986|
|[31.3895854806643...| 408.7354972658934|
|[31.4459724827577...| 481.7953656762313|
|[31.5171218025062...| 279.0853737340442|
|[31.5257524169682...| 449.1840246938907|
|[31.5741380228732...|   558.98158550018|
|[31.6253601348306...| 380.8996189793213|
|[31.6548096756927...|468.43826106249594|
|[31.7207699002873...| 545.2025339017355|
|[31.7216523605090...| 349.2909434580049|
|[31.8093003166791...| 548.1576442480632|
|[31.8209982016720...|416.63487334650745|
|[31.8279790554652...| 449.1186762202756|
|[31.9048571310136...|491.07764465663445|
|[31.9365486184489...| 440.7398697405149|
|[31.9453957483445...| 663.0011402051325|
+--------------------+------------

In [134]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 10.277797795777733
MSE: 105.63312753089365
